In [0]:
from functools import reduce
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
fact_fuel = spark.table("gov_br_combustivel.silver_layer.fact_fuel")

In [0]:
display(fact_fuel.schema)

## Variação mensal do ultimo ano

In [0]:

ultimo_ano = fact_fuel.agg(F.max("DataColeta")).collect()[0][0]

df_ultimo_ano = fact_fuel.filter(F.col("DataColeta") >= F.add_months(F.lit(ultimo_ano), -12))

df_mes = df_ultimo_ano.withColumn("year", F.year("DataColeta")) \
    .withColumn("month", F.month("DataColeta"))

media_mensal = df_mes.groupBy("Produto", "year", "month") \
    .agg(F.round(F.avg("ValorVenda"), 2).alias("avg_price")) \
    .orderBy("Produto", "year", "month")

windowSpec = Window.partitionBy("Produto").orderBy("year", "month")

view_variacao_mensal_produto = media_mensal.withColumn(
    "prev_price", F.lag("avg_price").over(windowSpec)
).withColumn(
    "variation_pct",
    F.round(((F.col("avg_price") - F.col("prev_price")) / F.col("prev_price")) * 100, 2)
)

In [0]:
view_variacao_mensal_produto.write.format("delta") \
    .mode("overwrite") \
    .option("zorderBy", "Produto") \
    .saveAsTable("gov_br_combustivel.gold_layer.view_variacao_mensal_produto")

### Preço médio por ano

In [0]:
df_com_ano = fact_fuel.withColumn("ano", F.year("DataColeta"))

media_anual = df_com_ano.groupBy("Produto", "ano") \
    .agg(F.round(F.avg("ValorVenda"), 2).alias("media_anual")) \
    .orderBy("Produto", "ano")

windowSpec = Window.partitionBy("Produto").orderBy("ano")

view_variacao_anual = media_anual.withColumn(
    "media_ano_anterior", F.lag("media_anual").over(windowSpec)
).withColumn(
    "yearly_variation_pct",
    F.round(((F.col("media_anual") - F.col("media_ano_anterior")) / F.col("media_ano_anterior")) * 100, 2)
)

In [0]:
primeiro_ano = fact_fuel.agg(F.min("DataColeta")).collect()[0][0].year

view_variacao_anual_sem_primeiro_ano = view_variacao_anual.filter(F.col("ano") > primeiro_ano)

In [0]:
view_variacao_anual_sem_primeiro_ano.write.format("delta") \
    .mode("overwrite") \
    .option("zorderBy", "Produto") \
    .saveAsTable("gov_br_combustivel.gold_layer.view_variacao_anual")